Mean Shift works on the following points:- (used for tracking a particular moving object)

1. We pass the initial location of our target object and the histogram back projected image to a mean shift function.

2. As the object moves, the histogram back projected image also changes.

3. The mean shift function moves the window to the new location with the max. probablity density.


Histogram Back Projected Image - It creates an image of the same size but of a single channel as of our input image, where each pixel corresponds to the probablity of that pixel belonging to our object i.e. the output image will have our object of intrest in more white color compared to the remaining part of that image.

Limitations of Mean Shift :-
 
1. Size of the target window does not change according to the movement of the object (ex: car window).

2. We have to give the initial postion of the region of intrest. 

In [ ]:
import numpy as np
import cv2 as cv

In [ ]:
cap = cv.VideoCapture('slow_traffic_small.mp4')

# take first frame of the video
ret, frame = cap.read()


# setup initial location of window
x, y, width, height = 300, 200, 100, 50
track_window = (x, y ,width, height)



# set up the ROI for tracking
roi = frame[y:y+height, x : x+width]
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255))) #np.array((0., 60., 32.) - lower bound and np.array((180., 255., 255)) - upper bound
# We use inRange function here to avoid the low light value or false value. 

roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180]) # [0] only 0 is used here because in histogram only hue is considered out of HSV
# [180] is the hist size 
# [0, 180] is the range

cv.normalize(roi_hist, roi_hist, 0, 255,cv.NORM_MINMAX)
# cv.normalize(source, destination, value of alpha , value of beta, norm type)





# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

cv.imshow('roi',roi)
while(1):
    ret, frame = cap.read()
    if ret == True:

        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # cv.calcBackProject(no. of images, channels, hist value, ranges, scale)
        
        # apply meanshift to get the new location
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        # cv.meanShift(destination image, tracked image, term criteria) 
        
        # Draw it on image
        x,y,w,h = track_window
        final_image = cv.rectangle(frame, (x,y), (x+w, y+h), 255, 3)

        cv.imshow('dst', dst)
        cv.imshow('final_image',final_image)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break